<a href="https://colab.research.google.com/github/Nikiloshen/Test-task-for-Quantum/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install simpletransformers

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [59]:
import pandas as pd
result = pd.read_csv('labeled_dataset.csv')
result['Word'] = result['Word'].apply(lambda x: str(x) if not pd.isnull(x) else '')

# Re-check the data types to confirm that "Word" is now all strings
print(result.dtypes)

# Save the cleaned dataset for inspection
result.to_csv("labeled_dataset_cleaned.csv", index=False)

Sentence #    object
Word          object
Tag           object
dtype: object


In [60]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from simpletransformers.ner import NERModel, NERArgs

data = pd.read_csv('labeled_dataset_cleaned.csv')
data['Sentence #'] = LabelEncoder().fit_transform(data['Sentence #'])

data.rename(columns={'Sentence #': 'sentence_id', 'Word': 'words', 'Tag': 'labels'}, inplace=True)

data['labels'] = data['labels'].str.upper()

X = data[['sentence_id', 'words']]
Y = data['labels']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Building up train and test data
train_data = pd.DataFrame({'sentence_id': x_train['sentence_id'], 'words': x_train['words'], 'labels': y_train})
test_data = pd.DataFrame({'sentence_id': x_test['sentence_id'], 'words': x_test['words'], 'labels': y_test})

train_data = train_data[train_data['words'].apply(lambda x: isinstance(x, str))]
test_data = test_data[test_data['words'].apply(lambda x: isinstance(x, str))]

"""Model Training"""
label = data['labels'].unique().tolist()

args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

model = NERModel('bert', 'bert-base-cased', labels=label, args=args)

model.train_model(train_data, eval_data=test_data, acc=accuracy_score)

result, model_outputs, preds_list = model.eval_model(test_data)
print(result)

prediction, model_output = model.predict(["Eh, how nice would it be to visit Mount Kilimanjaro? Have you ever been there? They say it's a really cool place."])
print(prediction)

# Save the Model
model.model.save_pretrained('model1')
model.tokenizer.save_pretrained('model1')
model.config.save_pretrained('model1/')

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/21 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_model.py:758: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 1:   0%|          | 0/1244 [00:00<?, ?it/s]

...

{'eval_loss': 0.09973293452371212, 'precision': 0.9488054607508533, 'recall': 0.7012909927288915, 'f1_score': 0.8064846416382252}


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_model.py:1643: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


[[{'Eh,': 'O'}, {'how': 'O'}, {'nice': 'O'}, {'would': 'O'}, {'it': 'O'}, {'be': 'O'}, {'to': 'O'}, {'visit': 'O'}, {'Mount': 'I-MOUNTAIN'}, {'Kilimanjaro?': 'B-MOUNTAIN'}, {'Have': 'O'}, {'you': 'O'}, {'ever': 'O'}, {'been': 'O'}, {'there?': 'O'}, {'They': 'O'}, {'say': 'O'}, {"it's": 'O'}, {'a': 'O'}, {'really': 'O'}, {'cool': 'O'}, {'place.': 'O'}]]


In [66]:
model.predict(["The Himalayan peak of Nanda-Devi is associated with the goddesses.", ])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'The': 'O'},
   {'Himalayan': 'B-MOUNTAIN'},
   {'peak': 'I-MOUNTAIN'},
   {'of': 'O'},
   {'Nanda-Devi': 'B-MOUNTAIN'},
   {'is': 'O'},
   {'associated': 'O'},
   {'with': 'O'},
   {'the': 'O'},
   {'goddesses.': 'O'}],
  [{'Mount': 'I-MOUNTAIN'},
   {'Ararat': 'O'},
   {'is': 'O'},
   {'a': 'O'},
   {'sacred': 'O'},
   {'mountain,': 'I-MOUNTAIN'},
   {'as': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'believed': 'O'},
   {'to': 'O'},
   {'be': 'O'},
   {'the': 'O'},
   {'landing': 'O'},
   {'place': 'O'},
   {'of': 'O'},
   {"Noah's": 'O'},
   {'Ark': 'O'}]],
 [[{'The': [[3.451, -1.909, -1.889]]},
   {'Himalayan': [[-0.343, 2.219, -1.646],
     [1.344, -0.5596, -0.2177],
     [3.133, -1.103, -1.942]]},
   {'peak': [[-2.053, -2.799, 4.7]]},
   {'of': [[4.75, -2.799, -2.018]]},
   {'Nanda-Devi': [[-2.488, 4.18, -1.238],
     [0.6235, 0.07965, -0.2656],
     [0.634, -0.0479, 0.3533],
     [0.7793, 0.7207, -0.73]]},
   {'is': [[5.914, -3.225, -3.12]]},
   {'associated': [[6.105, -3.7, 

In [48]:
def get_filtered_predictions(model, sentence):
    """
    Get filtered predictions from the model, replacing 'I-MOUNTAIN' with 'O' for the word 'mountain'
    if it is not preceded by a 'B-MOUNTAIN'.

    Args:
    model: The trained model for predictions.
    sentence (str): The input sentence for prediction.

    Returns:
    list: A list of tuples containing words and their corresponding labels after filtering.
    """
    # Perform predictions
    predictions = model.predict([sentence])[0]  # Get the first (and only) element from the outer list

    # List to hold filtered predictions
    filtered_predictions = []

    for i, word_dict in enumerate(predictions[0]):
        for word, label in word_dict.items():
            # Check if the word is "mountain"
            if word == 'mountain':
                # Check if the previous label was 'B-MOUNTAIN'
                if i > 0 and filtered_predictions and filtered_predictions[-1][1] == 'B-MOUNTAIN':
                    filtered_predictions.append((word, label))  # Keep 'I-MOUNTAIN'
                else:
                    filtered_predictions.append((word, 'O'))  # Change to 'O'
            else:
                filtered_predictions.append((word, label))

    return filtered_predictions

# Example usage
contextual_sentence = 'Mount Everest is the highest mountain on the Earth above sea level and is located in the  Mahalangur Himal sub-range of the Himalayas. The China–Nepal border runs across its summit point.'
filtered_output = get_filtered_predictions(model, contextual_sentence)

# Print filtered output
for word, label in filtered_output:
    print(word, label)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Mount O
Everest O
is O
the O
highest O
mountain O
on O
the O
Earth O
above O
sea O
level O
and O
is O
located O
in O
the O
Mahalangur O
Himal O
sub-range O
of O
the O
Himalayas. O
The O
China–Nepal O
border O
runs O
across O
its O
summit O
point. O


In [39]:
def extract_mountain_names(model, sentence):
    """
    Extract mountain names from the provided sentence using the model's predictions.

    Args:
    model: The trained model for predictions.
    sentence (str): The input sentence for prediction.

    Returns:
    list: A list of detected mountain names in the sentence.
    """
    # Get filtered predictions
    filtered_predictions = get_filtered_predictions(model, sentence)

    # List to hold detected mountain names
    mountain_names = []
    current_mountain = []

    for word, label in filtered_predictions:
        if label == 'B-MOUNTAIN':
            if current_mountain:  # If there is an ongoing mountain name, save it first
                mountain_names.append(' '.join(current_mountain))  # Join the current mountain name
                current_mountain = []  # Reset for new mountain name
            current_mountain.append(word)  # Start a new mountain name
        elif label == 'I-MOUNTAIN':
            current_mountain.append(word)  # Continue building the current mountain name
        else:
            if current_mountain:  # If we were building a mountain name
                mountain_names.append(' '.join(current_mountain))  # Save the current mountain name
                current_mountain = []  # Reset

    # If there's any mountain name left at the end, add it
    if current_mountain:
        mountain_names.append(' '.join(current_mountain))

    return mountain_names

# Example usage
input_sentence = input('Input your text: ')
mountain_names_output = extract_mountain_names(model, input_sentence)

# Print the detected mountain names
print("Detected mountain names:", mountain_names_output)

Input your text: a


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Detected mountain names: []
